In [ ]:
%load_ext autoreload
%autoreload 2
from synthpop.recipes.starter6 import Starter
starter_ver='6'

from output_verify import output_verify
from synthpop.synthesizer import synthesize_all, enable_logging 
import pandas as pd
import os,time
import pickle

outputfolder='/home/da/tools/synthpop/demos/outputs/'

# Synthesize by county 

In [ ]:
#for whole county
time0=time.time()
results=[]
def synthesize_counties(counties):
    
    for county in counties:
        pathout=outputfolder+county+"_starter%s_"%(starter_ver)
        starter = Starter(os.environ["CENSUS"], "MI", county)
        households, persons, fit, marginals=synthesize_all(starter)
        
        households.index.name='hh_id' #result[0] is hh
        households['serialno'] = households['serialno'].map(lambda x: '%f' % x)
        persons['serialno'] = persons['serialno'].map(lambda x: '%f' % x)
        #results[0].to_csv(outputfolder+county+"_starter%s_"%('3')+"HHs.csv",float_format='%.f') #float_format, not use scentific, pplies to all fp numbers 
        #could use results['serialno'] = results['serialno'].map(lambda x: '%f' % x) to change output format for serialno only
        households.to_csv(pathout+'_households.csv')
        persons.to_csv(pathout+'_persons.csv')
        
        df_verify=output_verify(households,persons,marginals)
        df_verify[0].to_csv(pathout+"_verify_households.csv")
        df_verify[1].to_csv(pathout+"_verify_persons.csv")
        df_verify[2].to_csv(pathout+"_verify_hh_size.csv")
        
        with open(pathout+'_fit_stats.pkl', 'wb') as f:
            pickle.dump(results[2], f)
        print time.time()-time0
    return results
#synthesize_counties(["Livingston County","Oakland County","Washtenaw County","Wayne County","Monroe County","Macomb County"])
res=synthesize_counties(["Macomb County"])


# Synthesize by selected BGs

In [ ]:
###for selected BGs
#enable_logging()
time0=time.time()
starter = Starter(os.environ["CENSUS"], "MI", 'Macomb County')
geoname="problembgs2.csv"
pathout=outputfolder+geoname+"_starter%s_"%(starter_ver)

dfbg=pd.read_csv(geoname,index_col=None,dtype=str) #make sure csv column in text not number, so read as'099' not 99
indd=[ind for i, ind in dfbg.iterrows()]
households, persons, fit, marginals = synthesize_all(starter, indexes=indd)
#HHs and Persons output
households.to_csv(pathout+'_households.csv')
persons.to_csv(pathout+'_persons.csv')

#save marginal and output comparison tables
df_veri=output_verify(households,persons,marginals)
df_veri[0].to_csv(pathout+"_verify_households.csv")
df_veri[1].to_csv(pathout+"_verify_persons.csv")
df_veri[2].to_csv(pathout+"_verify_hh_size.csv")
print time.time()-time0